In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler

import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Load your datasets
df = pd.read_csv('../../processed_data/full_data.csv')  





2024-11-15 18:13:28.322221: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 18:13:28.332776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731674608.346223   15120 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731674608.350376   15120 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 18:13:28.363651: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
# Step 1: Text Cleaning and Tokenization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [4]:
import re
def clean_text(text):
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    # Lemmatization
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return text

In [41]:
df['sub_category'].value_counts()

sub_category
upi related frauds                                                      35729
other                                                                   14547
debit/credit card fraud or sim swap fraud                               14357
internet banking related fraud                                          11844
fraud call/vishing                                                       7628
cyber bullying/stalking/sexting                                          5455
ewallet related fraud                                                    5385
rape/gang rape-sexually abusive content                                  3734
fakeimpersonating profile                                                3062
profile hacking identity theft                                           2823
cheating by impersonation                                                2706
sexually obscene material                                                2503
sexually explicit act                              

In [42]:
df.shape

(124887, 3)

In [43]:
df['crimeaditionalinfo'] = df['crimeaditionalinfo'].astype(str).apply(clean_text)


In [44]:
df['crimeaditionalinfo'].head()

0    continue received random call abusive message ...
1    fraudster continuously messaging asking pay mo...
2    acting like police demanding money adding sect...
3    apna job applied job interview telecalling res...
4    received call lady stating send new phone vivo...
Name: crimeaditionalinfo, dtype: object

In [56]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['crimeaditionalinfo'])

# Get all unique words (vocabulary)
unique_words = vectorizer.get_feature_names_out()

# Display the unique words
print("Total unique words:", len(unique_words))


Total unique words: 132377


In [57]:
X=df['crimeaditionalinfo']
y=df['sub_category']
# Encode category and subcategory labels
label_encoder_sub_category = LabelEncoder()
y_sub_category = label_encoder_sub_category.fit_transform(y)
ros = RandomOverSampler(sampling_strategy='not majority', random_state=42)
X_resampled, y_resampled = ros.fit_resample(X.values.reshape(-1, 1), y_sub_category)
X_resampled = X_resampled.flatten()

In [58]:

# Preprocess text data
max_words = 200000
max_len = 500
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X_resampled)

X_seq= tokenizer.texts_to_sequences(X_resampled)
X_pad = pad_sequences(X_seq, maxlen=max_len)


In [59]:
from tensorflow.keras.utils import to_categorical

y_sub_cat = to_categorical(y_resampled)


In [60]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_resampled),
    y=y_resampled
)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}


In [61]:
class_weights

{0: np.float64(1.0),
 1: np.float64(1.0),
 2: np.float64(1.0),
 3: np.float64(1.0),
 4: np.float64(1.0),
 5: np.float64(1.0),
 6: np.float64(1.0),
 7: np.float64(1.0),
 8: np.float64(1.0),
 9: np.float64(1.0),
 10: np.float64(1.0),
 11: np.float64(1.0),
 12: np.float64(1.0),
 13: np.float64(1.0),
 14: np.float64(1.0),
 15: np.float64(1.0),
 16: np.float64(1.0),
 17: np.float64(1.0),
 18: np.float64(1.0),
 19: np.float64(1.0),
 20: np.float64(1.0),
 21: np.float64(1.0),
 22: np.float64(1.0),
 23: np.float64(1.0),
 24: np.float64(1.0),
 25: np.float64(1.0),
 26: np.float64(1.0),
 27: np.float64(1.0),
 28: np.float64(1.0),
 29: np.float64(1.0),
 30: np.float64(1.0),
 31: np.float64(1.0),
 32: np.float64(1.0),
 33: np.float64(1.0),
 34: np.float64(1.0),
 35: np.float64(1.0),
 36: np.float64(1.0),
 37: np.float64(1.0),
 38: np.float64(1.0),
 39: np.float64(1.0),
 40: np.float64(1.0),
 41: np.float64(1.0)}

In [62]:
y_sub_cat.shape

(1500618, 42)

In [63]:
X_train, X_val, y_train, y_val = train_test_split(X_pad, y_sub_cat, test_size=0.2, random_state=42, stratify=y_resampled)



In [64]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
# Define the model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(max_words, 128, input_length=max_len)(input_layer)
lstm_layer = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense_layer = Dense(64, activation='relu')(lstm_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(len(np.unique(y_resampled)), activation='softmax')(dropout_layer)

# Compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(
    loss='categorical_crossentropy',  # Suitable for multiclass classification
    optimizer='adam',
    metrics=['accuracy']
)

# Display the model summary
model.summary()

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

/home/admin/Desktop/AI_Hackathon/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 500, 128)       │    25,600,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 42)             │         2,730 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,656,298 (97.87 MB)

 Trainable params: 25,656,298 (97.87 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
37516/37516 ━━━━━━━━━━━━━━━━━━━━ 13816s 368ms/step - accuracy: 0.5350 - loss: 1.4408 - val_accuracy: 0.7762 - val_loss: 0.5888
Epoch 2/5
37516/37516 ━━━━━━━━━━━━━━━━━━━━ 13616s 363ms/step - accuracy: 0.7747 - loss: 0.6012 - val_accuracy: 0.8036 - val_loss: 0.5003
Epoch 3/5
37516/37516 ━━━━━━━━━━━━━━━━━━━━ 13294s 354ms/step - accuracy: 0.7997 - loss: 0.5197 - val_accuracy: 0.8125 - val_loss: 0.4777
Epoch 4/5
37516/37516 ━━━━━━━━━━━━━━━━━━━━ 11204s 299ms/step - accuracy: 0.8088 - loss: 0.4919 - val_accuracy: 0.8167 - val_loss: 0.4652
Epoch 5/5
37516/37516 ━━━━━━━━━━━━━━━━━━━━ 11074s 295ms/step - accuracy: 0.8143 - loss: 0.4743 - val_accuracy: 0.8204 - val_loss: 0.4584
Validation Loss: 0.4584, Validation Accuracy: 0.8204


In [65]:
# Save the model in keras format

model.save('sub_category_text_classification_model.keras')


In [66]:
import pickle

with open('sub_category_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [67]:
# Save the label encoders to files
with open('label_encoder_sub_category.pickle', 'wb') as handle:
    pickle.dump(label_encoder_sub_category, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [2]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


max_words = 200000
max_len = 500

# Load the saved model
model = load_model('sub_category_text_classification_model.keras')

# Load the tokenizer
with open('sub_category_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoder
with open('label_encoder_sub_category.pickle', 'rb') as handle:
    label_encoder = pickle.load(handle)


2024-11-15 18:13:39.508772: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
# Sample new data
new_data = ["""hello sir and mam
this very urgent and emergency  to inform you
some one created a facebook fake account of my name and he demand for money of my facebook friends 
kindly request to you please take serious action on this matter"""]

# Clean and preprocess
new_data_cleaned = [clean_text(text) for text in new_data]

# Convert text to sequences
new_sequences = tokenizer.texts_to_sequences(new_data_cleaned)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len)  

In [8]:
# Make predictions
predictions = model.predict(new_padded_sequences)

# Convert predictions to label indices
predicted_indices = np.argmax(predictions, axis=1)

# Decode indices to original category labels
predicted_labels = label_encoder.inverse_transform(predicted_indices)

# Display results
for i, text in enumerate(new_data):
    print(f"Text: {new_data_cleaned}")
    print(f"Predicted Sub Category: {predicted_labels[i]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Text: ['hello sir mam urgent emergency inform one created facebook fake account name demand money facebook friend kindly request please take serious action matter']
Predicted Sub Category: fakeimpersonating profile
